In [9]:
import urllib.request
import urllib.error # Для обработки ошибок HTTP и URL
import zipfile
import os
import shutil # Для удаления директории, если нужно
import sys # Для sys.stdout.flush()
import time # Для задержки между попытками
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
import cv2
from PIL import Image

In [14]:

# Импортируем TensorFlow для предобработки, если он будет использоваться в нейросети
# Если вы используете PyTorch или другую библиотеку, замените на соответствующий импорт
# import tensorflow as tf 

# --- 1. Функция для безопасной загрузки изображения ---
def load_image_for_nn(image_path, target_size=(256, 256)):
    """
    Загружает изображение TIF, преобразует его в массив NumPy и выполняет предобработку.
    Возвращает массив изображения или None в случае ошибки.
    
    Аргументы:
    image_path (str): Путь к файлу изображения.
    target_size (tuple): Желаемый размер изображения (ширина, высота) для изменения размера.
                         Нейросети часто требуют фиксированный размер входных данных.
    """
    if not os.path.exists(image_path):
        print(f"Ошибка: Файл не найден по пути: {image_path}")
        return None

    try:
        # Открываем изображение с помощью Pillow
        img = Image.open(image_path)

        # Проверяем, что изображение успешно открыто
        if img is None:
            print(f"Ошибка: Не удалось открыть изображение по пути: {image_path}")
            return None

        # --- Шаг предобработки 1: Изменение размера изображения ---
        # Нейросети обычно требуют входные изображения фиксированного размера.
        # Используем Image.Resampling.LANCZOS для высококачественного изменения размера.
        img = img.resize(target_size, Image.Resampling.LANCZOS)

        # Преобразуем изображение в массив NumPy
        img_array = np.array(img)

        # --- Шаг предобработки 2: Нормализация пикселей ---
        # Для нейросетей часто требуется нормализация пикселей.
        # Если изображение в uint8 (0-255), нормализуем к диапазону 0-1.
        if img_array.dtype == np.uint8:
            img_array = img_array / 255.0
        # Если требуются другие виды нормализации (например, к диапазону -1 до 1), 
        # это можно добавить здесь:
        # img_array = (img_array - 0.5) * 2.0 
        
        # --- Шаг предобработки 3: Обработка каналов (если необходимо) ---
        # Убедитесь, что изображение имеет правильное количество каналов для вашей нейросети.
        # Например, если ожидается 3 канала (RGB), а изображение черно-белое (1 канал),
        # его можно преобразовать:
        # if len(img_array.shape) == 2: # Если это черно-белое изображение
        #     img_array = np.stack([img_array, img_array, img_array], axis=-1) # Преобразуем в 3 канала
        # elif len(img_array.shape) == 3 and img_array.shape[2] == 4: # Если есть альфа-канал (RGBA)
        #     img_array = img_array[:, :, :3] # Удаляем альфа-канал

        print(f"Изображение '{os.path.basename(image_path)}' успешно загружено и предобработано.")
        print(f"Новый размер изображения: {img.size}")
        print(f"Новая форма массива: {img_array.shape}")
        print(f"Тип данных массива после нормализации: {img_array.dtype}")

        return img_array

    except Exception as e:
        print(f"Произошла ошибка при загрузке или обработке файла {image_path}: {e}")
        return None

# --- 2. Функция для загрузки всех изображений из папки ---
def load_images_from_folder(folder_path, target_size=(256, 256)):
    """
    Загружает все TIF изображения из указанной папки и применяет предобработку.
    Возвращает словарь, где ключ - имя файла, значение - массив NumPy изображения.
    
    Аргументы:
    folder_path (str): Путь к папке с изображениями.
    target_size (tuple): Желаемый размер изображения (ширина, высота) для изменения размера.
    """
    if not os.path.isdir(folder_path):
        print(f"Ошибка: Папка не найдена по пути: {folder_path}")
        return {}
    loaded_images = {}
    print(f"Начинаем загрузку и предобработку изображений из папки: {folder_path}")
    
    # Получаем список всех файлов в папке
    for filename in os.listdir(folder_path):
        # Проверяем, что файл является TIF изображением
        if filename.lower().endswith('.tif'):
            file_path = os.path.join(folder_path, filename)
            print(f"Загрузка и предобработка файла: {filename}...")
            # Передаем target_size в функцию загрузки изображения
            img_data = load_image_for_nn(file_path, target_size)
            if img_data is not None:
                loaded_images[filename] = img_data
            else:
                print(f"Пропущено: {filename} из-за ошибки загрузки или предобработки.")
    
    print(f"Загружено и предобработано {len(loaded_images)} изображений из папки {folder_path}.")
    return loaded_images

# --- 3. Пути к вашим папкам ---
# Используйте "сырые" строки (r'...') для путей в Windows,
# чтобы избежать проблем с обратными слешами.
oil_folder_path = r'C:\Users\Sirius\Desktop\neuronetwork\01_Train_Val_Oil_Spill_images\Oil'
mask_folder_path = r'C:\Users\Sirius\Desktop\neuronetwork\01_Train_Val_Oil_Spill_images\Mask_oil'

# --- 4. Загрузка всех изображений из папок ---
# Определите желаемый размер для вашей нейросети.
# Пример: 256x256 пикселей.
IMAGE_SIZE = (256, 256) 

print("Попытка загрузки изображений из папки Oil:")
oil_images_data = load_images_from_folder(oil_folder_path, target_size=IMAGE_SIZE)

print("\nПопытка загрузки изображений из папки Mask_oil:")
mask_images_data = load_images_from_folder(mask_folder_path, target_size=IMAGE_SIZE)

# --- 5. Проверка и использование данных для нейросети ---
if oil_images_data:
    print("\nИзображения Oil готовы для обработки нейросетью.")
    # oil_images_data теперь содержит словарь с именами файлов и их предобработанными данными.
    # Вы можете перебрать их, например, для создания обучающих батчей:
    # for filename, img_array in oil_images_data.items():
    #     print(f"Обработка файла {filename} с формой {img_array.shape}")
    #     # Здесь вы можете передать img_array в вашу нейросеть.
    #     # Например, для модели, ожидающей батч:
    #     # input_for_model = np.expand_dims(img_array, axis=0) # Добавляем измерение для батча
    #     # prediction = model.predict(input_for_model)
else:
    print("\nНе удалось загрузить изображения из папки Oil. Нейросеть не может их обработать.")

if mask_images_data:
    print("Изображения Mask_oil готовы для обработки нейросетью.")
    # mask_images_data также содержит словарь с именами файлов и их предобработанными данными.
    # Эти данные будут служить целевыми масками для обучения, если вы делаете сегментацию.
else:
    print("Не удалось загрузить изображения из папки Mask_oil. Нейросеть не может их обработать.")

# --- Установка необходимых библиотек ---
# Если у вас еще нет этих библиотек, установите их через терминал VS Code:
# pip install Pillow numpy
# Если вы используете OpenCV (cv2), то:
# pip install opencv-python
# Если вы планируете использовать TensorFlow для дальнейшей работы с нейросетью:
# pip install tensorflow

Попытка загрузки изображений из папки Oil:
Начинаем загрузку и предобработку изображений из папки: C:\Users\Sirius\Desktop\neuronetwork\01_Train_Val_Oil_Spill_images\Oil
Загрузка и предобработка файла: 00000.tif...
Произошла ошибка при загрузке или обработке файла C:\Users\Sirius\Desktop\neuronetwork\01_Train_Val_Oil_Spill_images\Oil\00000.tif: cannot identify image file 'C:\\Users\\Sirius\\Desktop\\neuronetwork\\01_Train_Val_Oil_Spill_images\\Oil\\00000.tif'
Пропущено: 00000.tif из-за ошибки загрузки или предобработки.
Загрузка и предобработка файла: 00001.tif...
Произошла ошибка при загрузке или обработке файла C:\Users\Sirius\Desktop\neuronetwork\01_Train_Val_Oil_Spill_images\Oil\00001.tif: cannot identify image file 'C:\\Users\\Sirius\\Desktop\\neuronetwork\\01_Train_Val_Oil_Spill_images\\Oil\\00001.tif'
Пропущено: 00001.tif из-за ошибки загрузки или предобработки.
Загрузка и предобработка файла: 00002.tif...
Произошла ошибка при загрузке или обработке файла C:\Users\Sirius\Desktop

In [ ]:
dataset_url = "https://zenodo.org/records/8346860/files/01_Train_Val_Oil_Spill_images.7z?download=1"
output_filename = "01_Train_Val_Oil_Spill_images.7z" # Имя, под которым файл будет сохранен локально

# Директория для распакованных данных.
# Вы можете изменить это на любой путь на вашем компьютере, например, 'data/'
# или 'C:/Users/YourUser/Documents/MyDataset/'
unzip_directory = "C:\\Users\\Sirius\\Downloads"

# Параметры повторных попыток загрузки
max_retries = 5 # Максимальное количество попыток загрузки
retry_delay_seconds = 10 # Задержка между попытками в секундах

# --- Загрузка файла ---
print(f"Попытка загрузки датасета с: {dataset_url}")
print(f"Файл будет сохранен как: {output_filename}")

download_successful = False
for attempt in range(1, max_retries + 1):
    print(f"\nПопытка загрузки {attempt} из {max_retries}...")
    try:
        # Открываем URL для чтения
        with urllib.request.urlopen(dataset_url) as response:
            # Проверяем, что запрос был успешным (код 200)
            if response.getcode() != 200:
                raise urllib.error.HTTPError(dataset_url, response.getcode(),
                                             response.reason, response.headers, None)

            # Получаем общий размер файла, если он доступен
            total_size = int(response.headers.get('Content-Length', 0))
            downloaded_size = 0
            block_size = 8192 # Размер блока для чтения (8 КБ)

            # Открываем локальный файл для записи в бинарном режиме
            with open(output_filename, 'wb') as out_file:
                while True:
                    buffer = response.read(block_size)
                    if not buffer:
                        break # Если буфер пуст, значит, файл прочитан до конца
                    out_file.write(buffer)
                    downloaded_size += len(buffer)

                    # Выводим прогресс загрузки
                    if total_size > 0:
                        progress = (downloaded_size / total_size) * 100
                        # Используем \r для перезаписи строки и end='' для предотвращения новой строки
                        sys.stdout.write(f"\rЗагружено: {downloaded_size / (1024*1024):.2f} MB / {total_size / (1024*1024):.2f} MB ({progress:.2f}%)")
                        sys.stdout.flush() # Принудительно выводим буфер stdout
                    else:
                        # Если размер файла неизвестен, просто показываем загруженный объем
                        sys.stdout.write(f"\rЗагружено: {downloaded_size / (1024*1024):.2f} MB")
                        sys.stdout.flush()

        print(f"\nЗагрузка '{output_filename}' завершена.")
        download_successful = True
        break # Выходим из цикла повторных попыток, если загрузка успешна

    except urllib.error.HTTPError as e:
        print(f"\nОшибка HTTP при загрузке файла (код: {e.code}, причина: {e.reason}): {e}")
    except urllib.error.URLError as e:
        print(f"\nОшибка URL при загрузке файла (причина: {e.reason}): {e}")
    except Exception as e:
        print(f"\nПроизошла непредвиденная ошибка при загрузке файла: {e}")

    # Если загрузка не удалась, удаляем частично загруженный файл перед следующей попыткой
    if os.path.exists(output_filename):
        os.remove(output_filename)
        print(f"Удален частично загруженный файл '{output_filename}'.")
    if attempt < max_retries:
        print(f"Повторная попытка через {retry_delay_seconds} секунд...")
        time.sleep(retry_delay_seconds)
    else:
        print(f"Все {max_retries} попыток загрузки не удались. Выход.")
        sys.exit(1) # Выходим с кодом ошибки

if not download_successful:
    sys.exit(1) # Выходим, если загрузка не была успешной после всех попыток

# --- Проверка размера загруженного файла ---
if os.path.exists(output_filename):
    file_size_bytes = os.path.getsize(output_filename)
    print(f"Размер загруженного файла '{output_filename}': {file_size_bytes / (1024*1024*1024):.2f} GB")
else:
    print(f"Ошибка: Файл '{output_filename}' не был загружен.")
    sys.exit(1)

# --- Распаковка архива ---
# Сначала создадим директорию для распакованных данных, если ее нет.
os.makedirs(unzip_directory, exist_ok=True)

print(f"\nНачинаем распаковку '{output_filename}' в '{unzip_directory}' (это может занять много времени)...")

try:
    # Проверяем, является ли файл ZIP-архивом
    if zipfile.is_zipfile(output_filename):
        with zipfile.ZipFile(output_filename, 'r') as zip_ref:
            # Распаковываем все содержимое архива в указанную директорию
            zip_ref.extractall(unzip_directory)
        print(f"Распаковка '{output_filename}' завершена.")
    else:
        print(f"Ошибка: '{output_filename}' не является действительным ZIP-архивом. Проверьте формат файла.")
        # Если это tar.gz, используйте:
        # import tarfile
        # with tarfile.open(output_filename, "r:gz") as tar_ref:
        #     tar_ref.extractall(unzip_directory)

except zipfile.BadZipFile:
    print(f"Ошибка: '{output_filename}' поврежден или не является действительным ZIP-архивом.")
except Exception as e:
    print(f"Произошла ошибка при распаковке: {e}")
    sys.exit(1)

print(f"\nСодержимое распакованной папки '{unzip_directory}':")
# Выводим список файлов в распакованной директории
for item in os.listdir(unzip_directory):
    print(f"- {item}")

In [ ]:
DATA_DIR = 'C:\\Users\\Sirius\\Desktop\\neuronetwork\\GOTOVO' 
IMAGE_SUBDIR = 'processed_oil_spill_images' # Поддиректория, где хранятся двухканальные изображения
MASK_SUBDIR ="processed_oil_spill_images" # Поддиректория, где хранятся пиксельные маски

IMAGE_HEIGHT = 256 
IMAGE_WIDTH = 256  
NUM_CHANNELS = 2048   
NUM_CLASSES = 1    


# --- 2. Загрузка и предварительная обработка данных ---

def load_image_and_mask(image_path, mask_path, target_size=(IMAGE_HEIGHT, IMAGE_WIDTH)):
    """
    Загружает двухканальное изображение и соответствующую пиксельную маску.
    Вам нужно будет адаптировать эту функцию под формат ваших файлов.
    Например, если ваши двухканальные изображения хранятся как отдельные файлы
    для каждого канала или как многоканальные TIFF.
    """
    try:
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED) # Загрузка без изменений, сохраняя каналы
        if image.shape[-1] != NUM_CHANNELS:
            print(f"Предупреждение: Изображение {image_path} имеет {image.shape[-1]} каналов, ожидалось {NUM_CHANNELS}.")
        # Загрузка маски 
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Изменение размера
        image = cv2.resize(image, target_size)
        mask = cv2.resize(mask, target_size, interpolation=cv2.INTER_NEAREST) # Для масок используем INTER_NEAREST, чтобы сохранить дискретные значения

        # Нормализация изображений (0-1)
        image = image.astype(np.float32) / 255.0
        # Нормализация маски (0 или 1 для бинарной маски)
        mask = mask.astype(np.float32) / 255.0
        mask = np.expand_dims(mask, axis=-1) 

        return image, mask
    except Exception as e:
        print(f"Ошибка при загрузке или обработке {image_path} или {mask_path}: {e}")
        return None, None
def load_dataset(data_dir, image_subdir, mask_subdir):
    """
    Собирает пути ко всем изображениям и маскам в датасете.
    """
    image_paths = sorted([os.path.join(data_dir, image_subdir, f) for f in os.listdir(os.path.join(data_dir, image_subdir)) if f.endswith(('.png', '.tif', '.jpg'))])
    mask_paths = sorted([os.path.join(data_dir, mask_subdir, f) for f in os.listdir(os.path.join(data_dir, mask_subdir)) if f.endswith(('.png', '.tif', '.jpg'))])

    # Убедитесь, что количество изображений и масок совпадает
    if len(image_paths) != len(mask_paths):
        raise ValueError("Количество изображений и масок не совпадает!")
    images = []
    masks = []
    for i in range(len(image_paths)):
        img, msk = load_image_and_mask(image_paths[i], mask_paths[i])
        if img is not None and msk is not None:
            images.append(img)
            masks.append(msk)

    return np.array(images), np.array(masks)

print("Загрузка данных...")
try:
    X, y = load_dataset(DATA_DIR, IMAGE_SUBDIR, MASK_SUBDIR)
    print(f"Загружено {len(X)} изображений и масок.")
    print(f"Форма изображений: {X.shape}") # Ожидается (количество_образцов, высота, ширина, количество_каналов)
    print(f"Форма масок: {y.shape}")     # Ожидается (количество_образцов, высота, ширина, 1)

    # Разделение данных на обучающую и валидационную выборки
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Обучающая выборка: {X_train.shape}, {y_train.shape}")
    print(f"Валидационная выборка: {X_val.shape}, {y_val.shape}")

except Exception as e:
    print(f"Произошла ошибка при загрузке или разделении данных: {e}")
    print("Пожалуйста, проверьте пути к файлам и формат ваших данных.")
    # Выход из программы или использование заглушечных данных для демонстрации модели
    X_train = np.random.rand(10, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS)
    y_train = np.random.randint(0, 2, size=(10, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CLASSES))
    X_val = np.random.rand(2, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS)
    y_val = np.random.randint(0, 2, size=(2, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CLASSES))
    print("Используются заглушечные данные для продолжения демонстрации модели.")


# --- 3. Определение архитектуры сверточной нейросети (U-Net-подобная) ---

def unet_model(input_size=(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS), num_classes=NUM_CLASSES):
    inputs = keras.Input(input_size)

    # Encoder (Путь сжатия)
    conv1 = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(32, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(64, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck (Дно)
    conv5 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv5)
# Decoder (Путь расширения)
    up6 = layers.UpSampling2D(size=(2, 2))(conv5)
    up6 = layers.Conv2D(256, 2, activation='relu', padding='same')(up6)
    merge6 = layers.concatenate([conv4, up6], axis=3) # Skip connection
    conv6 = layers.Conv2D(256, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv6)

    up7 = layers.UpSampling2D(size=(2, 2))(conv6)
    up7 = layers.Conv2D(128, 2, activation='relu', padding='same')(up7)
    merge7 = layers.concatenate([conv3, up7], axis=3) # Skip connection
    conv7 = layers.Conv2D(128, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv7)

    up8 = layers.UpSampling2D(size=(2, 2))(conv7)
    up8 = layers.Conv2D(64, 2, activation='relu', padding='same')(up8)
    merge8 = layers.concatenate([conv2, up8], axis=3) # Skip connection
    conv8 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv8)
    
    up9 = layers.UpSampling2D(size=(2, 2))(conv8)
    up9 = layers.Conv2D(32, 2, activation='relu', padding='same')(up9)
    merge9 = layers.concatenate([conv1, up9], axis=3) # Skip connection
    conv9 = layers.Conv2D(32, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv2D(32, 3, activation='relu', padding='same')(conv9)
    
    
    # Для бинарной сегментации (один класс маски):
    if num_classes == 1:
        outputs = layers.Conv2D(num_classes, 1, activation='sigmoid')(conv9)
        loss_function = 'binary_crossentropy'
    # Для многоклассовой сегментации:
    else:
        outputs = layers.Conv2D(num_classes, 1, activation='softmax')(conv9)
        loss_function = 'categorical_crossentropy'

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model, loss_function

print("Создание модели нейросети...")
model, loss_func = unet_model()
model.summary()

# --- 4. Компиляция модели ---
print("Компиляция модели...")
model.compile(optimizer='adam', loss=loss_func, metrics=['accuracy'])

# --- 5. Обучение модели ---
print("Обучение модели...")
# Вы можете настроить количество эпох и размер батча
EPOCHS = 100
BATCH_SIZE = 10

history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val, y_val),
    verbose=1
)


print("\nОценка модели на валидационных данных:")
loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"Потери на валидационной выборке: {loss:.4f}")
print(f"Точность на валидационной выборке: {accuracy:.4f}")

Загрузка данных...
Предупреждение: Изображение C:\Users\Sirius\Desktop\neuronetwork\GOTOVO\processed_oil_spill_images\processed_00000.tif имеет 2048 каналов, ожидалось 1.
Предупреждение: Изображение C:\Users\Sirius\Desktop\neuronetwork\GOTOVO\processed_oil_spill_images\processed_00001.tif имеет 2048 каналов, ожидалось 1.
Предупреждение: Изображение C:\Users\Sirius\Desktop\neuronetwork\GOTOVO\processed_oil_spill_images\processed_00002.tif имеет 2048 каналов, ожидалось 1.
Предупреждение: Изображение C:\Users\Sirius\Desktop\neuronetwork\GOTOVO\processed_oil_spill_images\processed_00003.tif имеет 2048 каналов, ожидалось 1.
Предупреждение: Изображение C:\Users\Sirius\Desktop\neuronetwork\GOTOVO\processed_oil_spill_images\processed_00004.tif имеет 2048 каналов, ожидалось 1.
Предупреждение: Изображение C:\Users\Sirius\Desktop\neuronetwork\GOTOVO\processed_oil_spill_images\processed_00005.tif имеет 2048 каналов, ожидалось 1.
Предупреждение: Изображение C:\Users\Sirius\Desktop\neuronetwork\GOTO

KeyboardInterrupt: 